# Minibatch Stochastic Gradient Descent

Trong học sâu, việc tối ưu hóa các tham số của mô hình là một bước quan trọng để đạt được hiệu suất tốt nhất. Có nhiều phương pháp tối ưu hóa khác nhau, trong đó Gradient Descent (GD) và Stochastic Gradient Descent (SGD) là hai phương pháp phổ biến. Tuy nhiên, cả hai đều có những hạn chế riêng. Minibatch Stochastic Gradient Descent (Minibatch SGD) được xem là một giải pháp cân bằng giữa hai phương pháp này, mang lại hiệu quả cả về mặt tính toán và thống kê.

## Vectorization and Caches

Trọng tâm của quyết định sử dụng minibatches là hiệu quả tính toán. Điều này dễ hiểu nhất khi xem xét song song với nhiều GPU và nhiều máy chủ. Trong trường hợp này, chúng ta cần gửi ít nhất một hình ảnh cho mỗi GPU. Với 8 GPU trên mỗi máy chủ và 16 máy chủ, chúng tôi đã có kích thước minibatch là 128. 

Mọi thứ tinh tế hơn một chút khi nói đến GPU đơn lẻ hoặc thậm chí CPU. Các thiết bị này có nhiều loại bộ nhớ, thường là nhiều loại đơn vị tính toán và hạn chế băng thông khác nhau giữa chúng. Ví dụ, CPU có một số lượng nhỏ các thanh ghi và sau đó là L1, L2 và trong một số trường hợp thậm chí bộ nhớ cache L3 (được chia sẻ giữa các lõi bộ xử lý khác nhau). Các bộ nhớ đệm này có kích thước và độ trễ ngày càng tăng (đồng thời chúng giảm băng thông). Nó đủ để nói, bộ xử lý có khả năng thực hiện nhiều hoạt động hơn so với những gì giao diện bộ nhớ chính có thể cung cấp. 

* CPU 2GHz với 16 lõi và vectorization AVX-512 có thể xử lý lên đến $2 \cdot 10^9 \cdot 16 \cdot 32 = 10^{12}$ byte mỗi giây. Khả năng của GPU dễ dàng vượt quá con số này theo hệ số 100. Mặt khác, một bộ xử lý máy chủ tầm trung có thể không có nhiều hơn 100 Gb/s băng thông, tức là, ít hơn một phần mười những gì sẽ được yêu cầu để giữ cho bộ xử lý ăn. Để làm cho vấn đề tồi tệ hơn, không phải tất cả truy cập bộ nhớ được tạo ra bằng nhau: đầu tiên, giao diện bộ nhớ thường rộng 64 bit hoặc rộng hơn (ví dụ, trên GPU lên đến 384 bit), do đó đọc một byte duy nhất phải chịu chi phí truy cập rộng hơn nhiều.
* Có chi phí đáng kể cho truy cập đầu tiên trong khi truy cập tuần tự là tương đối rẻ (điều này thường được gọi là một lần đọc liên tục). Có rất nhiều điều cần lưu ý, chẳng hạn như bộ nhớ đệm khi chúng ta có nhiều ổ cắm, chiplet và các cấu trúc khác. Một cuộc thảo luận chi tiết về điều này nằm ngoài phạm vi của phần này. Xem ví dụ, [Wikipedia article](https://en.wikipedia.org/wiki/Cache_hierarchy) này để có một cuộc thảo luận chuyên sâu hơn.

Cách để giảm bớt những hạn chế này là sử dụng một hệ thống phân cấp của bộ nhớ cache CPU thực sự đủ nhanh để cung cấp cho bộ xử lý dữ liệu. Đây là * động lực là* đằng sau việc phân mẻ trong học sâu. Để giữ cho vấn đề đơn giản, hãy xem xét phép nhân ma trận ma trận, nói $\mathbf{A} = \mathbf{B}\mathbf{C}$. Chúng tôi có một số tùy chọn để tính toán $\mathbf{A}$. Ví dụ, chúng tôi có thể thử như sau: 

1. Chúng tôi có thể tính toán $\mathbf{A}_{ij} = \mathbf{B}_{i,:} \mathbf{C}_{:,j}^\top$, tức là, chúng tôi có thể tính toán nó elementwise bằng phương tiện của các sản phẩm chấm.
1. Chúng ta có thể tính toán $\mathbf{A}_{:,j} = \mathbf{B} \mathbf{C}_{:,j}^\top$, tức là, chúng ta có thể tính toán nó một cột tại một thời điểm. Tương tự như vậy chúng ta có thể tính $\mathbf{A}$ một hàng $\mathbf{A}_{i,:}$ tại một thời điểm.
1. We could simply đơn giản compute tính toán $\mathbf{A} = \mathbf{B} \mathbf{C}$.
1. Chúng ta có thể phá vỡ $\mathbf{B}$ và $\mathbf{C}$ thành các ma trận khối nhỏ hơn và tính toán $\mathbf{A}$ một khối tại một thời điểm.

Nếu chúng ta làm theo tùy chọn đầu tiên, chúng ta sẽ cần sao chép một hàng và một vectơ cột vào CPU mỗi khi chúng ta muốn tính một phần tử $\mathbf{A}_{ij}$. Thậm chí tệ hơn, do thực tế là các yếu tố ma trận được căn chỉnh tuần tự, do đó, chúng ta được yêu cầu truy cập nhiều vị trí tách rời cho một trong hai vectơ khi chúng ta đọc chúng từ bộ nhớ. Tùy chọn thứ hai thuận lợi hơn nhiều. Trong đó, chúng ta có thể giữ vector cột $\mathbf{C}_{:,j}$ trong bộ nhớ cache CPU trong khi chúng tôi tiếp tục đi qua $B$. Điều này giảm một nửa yêu cầu băng thông bộ nhớ với truy cập nhanh hơn tương ứng. Tất nhiên, tùy chọn 3 là mong muốn nhất. Thật không may, hầu hết các ma trận có thể không hoàn toàn phù hợp với bộ nhớ cache (đây là những gì chúng ta đang thảo luận sau khi tất cả). Tuy nhiên, tùy chọn 4 cung cấp một giải pháp thay thế thực tế hữu ích: chúng ta có thể di chuyển các khối ma trận vào bộ nhớ cache và nhân chúng cục bộ. Tối ưu hóa thư viện chăm sóc này cho chúng tôi. Chúng ta hãy xem các hoạt động này hiệu quả như thế nào trong thực tế. 

Ngoài hiệu quả tính toán, chi phí được giới thiệu bởi Python và bởi chính khuôn khổ học tập sâu là đáng kể. Nhớ lại rằng mỗi lần chúng ta thực hiện một lệnh, trình thông dịch Python sẽ gửi một lệnh đến công cụ MXNet cần chèn nó vào biểu đồ tính toán và xử lý nó trong quá trình lên lịch. Chi phí như vậy có thể khá bất lợi. Nói tóm lại, rất nên sử dụng vector hóa (và ma trận) bất cứ khi nào có thể.

In [ ]:
import time
import numpy as np
import torch
from torch import nn
import d2l

A = torch.zeros(512, 512)
B = torch.randn(512, 512)
C = torch.randn(512, 512)

Vì chúng ta sẽ thường xuyên đo thời gian chạy trong phần còn lại của báo cáo, hãy định nghĩa một bộ đếm thời gian.

In [ ]:
class Timer:
    """Record multiple running times."""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """Start the timer."""
        self.tik = time.time()

    def stop(self):
        """Stop the timer and record the time in a list."""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """Return the average time."""
        return sum(self.times) / len(self.times)

    def sum(self):
        """Return the sum of time."""
        return sum(self.times)

    def cumsum(self):
        """Return the accumulated time."""
        return np.array(self.times).cumsum().tolist()

timer = Timer()

Gán từng phần tử đơn giản là lặp qua tất cả các hàng và cột của $\mathbf{B}$ và $\mathbf{C}$ tương ứng để gán giá trị cho $\mathbf{A}$.

In [ ]:
# Compute A = BC one element at a time
timer.start()
for i in range(512):
    for j in range(512):
        A[i, j] = torch.dot(B[i, :], C[:, j])
timer.stop()

Một chiến lược nhanh hơn là thực hiện gán theo cột.

In [ ]:
# Compute A = BC one column at a time
timer.start()
for j in range(512):
    A[:, j] = torch.mv(B, C[:, j])
timer.stop()

Cuối cùng, phương pháp hiệu quả nhất là thực hiện toàn bộ phép toán trong một khối duy nhất.
Lưu ý rằng việc nhân hai ma trận $\mathbf{B} \in \mathbb{R}^{m \times n}$ và $\mathbf{C} \in \mathbb{R}^{n \times p}$ cần khoảng $2mnp$ phép toán dấu phẩy động,
khi phép nhân và cộng vô hướng được tính là hai phép toán riêng biệt (mặc dù trên thực tế thường được gộp lại).
Do đó, việc nhân hai ma trận kích thước $512 \times 512$ cần khoảng $0.27$ tỷ phép toán dấu phẩy động.
Bây giờ, hãy cùng xem tốc độ tương ứng của các phép toán này.

In [ ]:
# Compute A = BC in one go
timer.start()
A = torch.mm(B, C)
timer.stop()

gigaflops = [0.27 / i for i in timer.times]
print(f'performance in Gigaflops: element {gigaflops[0]:.3f}, '
      f'column {gigaflops[1]:.3f}, full {gigaflops[2]:.3f}')

## Minibatches

Trong quá khứ, chúng tôi đã cho rằng chúng tôi sẽ đọc * minibatches* của dữ liệu chứ không phải là quan sát duy nhất để cập nhật các tham số. Bây giờ chúng tôi đưa ra một lời biện minh ngắn gọn cho nó. Xử lý các quan sát đơn yêu cầu chúng ta thực hiện nhiều phép nhân ma trận-vector (hoặc thậm chí là vector-vector) đơn lẻ, khá tốn kém và phát sinh một chi phí đáng kể thay mặt cho khuôn khổ học sâu cơ bản. Điều này áp dụng cả để đánh giá một mạng khi áp dụng cho dữ liệu (thường được gọi là suy luận) và khi tính toán gradient để cập nhật các tham số. Đó là, điều này áp dụng bất cứ khi nào chúng tôi thực hiện $\mathbf{w} \leftarrow \mathbf{w} - \eta_t \mathbf{g}_t$ ở đâu 

$$\mathbf{g}_t = \partial_{\mathbf{w}} f(\mathbf{x}_{t}, \mathbf{w})$$

Chúng ta có thể tăng hiệu quả * tính tị* của thao tác này bằng cách áp dụng nó vào một loạt các quan sát tại một thời điểm. Đó là, chúng tôi thay thế gradient $\mathbf{g}_t$ trong một quan sát duy nhất bằng một trong một lô nhỏ 

$$\mathbf{g}_t = \partial_{\mathbf{w}} \frac{1}{|\mathcal{B}_t|} \sum_{i \in \mathcal{B}_t} f(\mathbf{x}_{i}, \mathbf{w})$$

Chúng ta hãy xem điều này làm gì với các thuộc tính thống kê của $\mathbf{g}_t$: vì cả $\mathbf{x}_t$ và tất cả các yếu tố của minibatch $\mathcal{B}_t$ được vẽ đồng đều ngẫu nhiên từ bộ đào tạo, kỳ vọng của gradient vẫn không thay đổi. Mặt khác, phương sai được giảm đáng kể. Kể từ khi gradient minibatch bao gồm $b := |\mathcal{B}_t|$ gradient độc lập đang được trung bình, độ lệch chuẩn của nó được giảm bởi một hệ số $b^{-\frac{1}{2}}$. Bản thân nó, điều này là một điều tốt, vì nó có nghĩa là các bản cập nhật được liên kết đáng tin cậy hơn với gradient đầy đủ. 

Ngây thơ điều này sẽ chỉ ra rằng việc lựa chọn một minibatch lớn $\mathcal{B}_t$ sẽ là mong muốn phổ biến. Than ôi, sau một số điểm, việc giảm thêm độ lệch chuẩn là tối thiểu khi so sánh với sự gia tăng tuyến tính trong chi phí tính toán. Trong thực tế, chúng tôi chọn một minibatch đủ lớn để mang lại hiệu quả tính toán tốt trong khi vẫn phù hợp với bộ nhớ của GPU. Để minh họa tiết kiệm chúng ta hãy xem xét một số mã. Trong đó chúng ta thực hiện cùng một phép nhân ma trận ma trận, nhưng lần này chia thành “minibatches” của 64 cột tại một thời điểm.

In [ ]:
timer.start()
for j in range(0, 512, 64):
    A[:, j:j+64] = torch.mm(B, C[:, j:j+64])
timer.stop()
print(f'performance in Gigaflops: block {0.27 / timer.times[3]:.3f}')

Có thể thấy quá trình tính toán trên minibatch về cơ bản có hiệu suất gần bằng thực hiện trên toàn ma trận. Tuy nhiên, cần lưu ý rằng Trong ví dụ trước đó ta sử dụng một loại điều chuẩn phụ thuộc chặt chẽ vào phương sai của minibatch. khi tăng kích thước minibatch, phương sai giảm xuống và cùng với đó là lợi ích của việc thêm nhiễu (noise-injection) cũng giảm theo do phương pháp chuẩn hóa theo batch.

## Reading the Dataset

Chúng ta hãy xem cách minibatches được tạo ra hiệu quả từ dữ liệu. Sau đây chúng tôi sử dụng một tập dữ liệu do NASA phát triển để kiểm tra cánh [noise from different aircraft](https://archive.ics.uci.edu/ml/datasets/Airfoil+Self-Noise) để so sánh các thuật toán tối ưu hóa này. Để thuận tiện, chúng tôi chỉ sử dụng các ví dụ $1,500$ đầu tiên. Dữ liệu được làm trắng để xử lý trước, tức là, chúng tôi loại bỏ trung bình và giải thích phương sai thành $1$ cho mỗi tọa độ.

In [ ]:
#@save
d2l.DATA_HUB['airfoil'] = (d2l.DATA_URL + 'airfoil_self_noise.dat',
                           '76e5be1548fd8222e5074cf0faae75edff8cf93f')

#@save
def get_data_ch11(batch_size=10, n=1500):
    data = np.genfromtxt(d2l.download('airfoil'),
                         dtype=np.float32, delimiter='\t')
    data = torch.from_numpy((data - data.mean(axis=0)) / data.std(axis=0))
    data_iter = d2l.load_array((data[:n, :-1], data[:n, -1]),
                               batch_size, is_train=True)
    return data_iter, data.shape[1] - 1

## Hiện thực từ đầu

Để so sánh hiệu năng giữa 3 thuật toán tối ưu là GD, SGD và SGD theo minibatch, ta sẽ sử dụng mô hình Hồi quy Tuyến tính

### Nhắc lại về Hồi quy Tuyến tính

#### Định nghĩa
Hồi quy Tuyến tính (Linear Regression) là một phương pháp học máy cơ bản thuộc nhóm học có giám sát (Supervised learning). Phương pháp này bao gồm 2 yếu tố là Hồi quy và Tuyến tính. Phép Hồi quy phân tích mối quan hệ giữa một hoặc nhiều biến đầu vào (biến độc lập) và một biến đầu ra (biến phụ thuộc). Trong khi đó, phép Tuyến tính chỉ mối quan hệ tổ hợp tuyến tính của các biến đầu vào mà không có sự xuất hiện của các hàm lũy thừa hay phi tuyến như sin, log, relu, ... Như vậy, Hồi quy Tuyến tính là phương pháp học máy và thống kê giúp mô hình hóa mối quan hệ **tuyến tính** giữa một hoặc nhiều biến đầu vào (biến độc lập) và một biến đầu ra (biến phụ thuộc), từ đó giúp dự đoán giá trị của biến phụ thuộc dựa trên giá trị của các biến độc lập.

#### Mô hình Tuyến tính
Mô hình Hồi quy Tuyến tính cho `d` đặc trưng (biến đầu vào) có dạng:
$$
y = \omega_1*x_1 + \omega_2*x_2 + ... + \omega_d*x_d + b
$$
Thu thập toàn bộ các đặc trưng vào một vector $\mathbf{x}$ và toàn bộ các trọng số vào một vector $\boldsymbol{\omega}$, ta có thể biểu diễn mô hình dưới dạng tích vô hướng của 2 vector:
$$
y = \boldsymbol{\omega}^T*\mathbf{x}
$$
Trong đó:
- $\mathbf{x}$: vector đặc trưng đầu vào, $\mathbf{x} \in R^d$ (input features)
- $\boldsymbol{\omega}$: vector trọng số (weights) cần huấn luyện
- $b$: độ lệch (bias)
- $y$: giá trị đầu ra (output)


#### Hàm mất mát
Để đánh giá mức độ khớp giữa mô hình được xây dựng và dữ liệu, ta sử dụng hàm mất mát. Hàm mất mát định lượng khoảng cách giữa giá trị thực $y$ và giá trị dự đoán $\hat{y}$ của mục tiêu. Độ mất mát thường là một số không âm và có giá trị càng nhỏ càng tốt. Khi các dự đoán hoàn hảo, chúng sẽ có độ mất mát sẽ bằng **0**. Hàm mất mát thông dụng nhất trong các bài toán hồi quy là hàm tổng bình phương các lỗi - Mean Squared Error (MSE):
$$
L_i = \frac{1}{2} MSE(y_i, \hat{y}_i) = \frac{1}{2} E[(y_i - \hat{y}_i)^2] = \frac{1}{2} (y_i - \hat{y}_i)^2
$$
Hằng số **1/2** không tạo ra sự khác biệt thực sự nào nhưng sẽ giúp ký hiệu thuận tiện hơn:nó sẽ được triệt tiêu khi lấy đạo hàm của hàm mất mát.

Lưu ý rằng khi hiệu giữa giá trị thực $y_i$ và giá trị ước lượng $\hat{y}_i$ lớn, giá trị hàm mất mát sẽ tăng rất lớn cho sự phụ thuộc bậc 2. Để đo chất lượng của mô hình trên toàn bộ tập dữ liệu, ta đơn thuần lấy trung bình (hay tương đương là lấy tổng) các giá trị mất mát của từng mẫu trong tập huấn luyện.
$$
L = \frac{1}{n} \sum_{i=1}^{n} L_i = \frac{1}{n} \sum_{i=1}^{n} \frac{1}{2} (y_i - \hat{y}_i)^2
$$


#### Mục tiêu
Khi huấn luyện mô hình, ta muốn tìm các tham số $\boldsymbol{\omega}^*$ và $b^*$ sao cho tổng độ mất mát trên toàn bộ các mẫu huấn luyện được cực tiểu hóa:
$$
\boldsymbol{\omega}^*, b^* = \underset{\boldsymbol{\omega}, b}{\text{argmin}}\:L(\boldsymbol{\omega}, b)
$$

Kỹ thuật chính để tối ưu hóa mô hình này, cũng như các mô hình học sâu khác, bao gồm việc giảm thiểu lỗi qua các vòng lặp bằng cách cập nhật tham số theo hướng làm giảm gần hàm mát mát. Với các hàm mất mát mặt lồi, giá trị mất mát cuối cùng sẽ hội tụ về giá trị nhỏ nhất. Tuy điều tương tự không thể áp dụng cho các mặt không lồi, ít nhất thuật toán sẽ dẫn tới một cực tiểu (hy vọng là tốt)

Đơn giản nhất ta có thể kể đến là việc tính đạo hàm của hàm mất mát, tức trung bình của các giá trị mất mát được tính trên mỗi mẫu của tập dữ liệu. Cuối cùng, gradient này được nhân với tốc độ học $\eta > 0$ và kết quả này được trừ đi từ các giá trị tham số hiện tại. Đây là chính phương pháp Gradient Descent (GD).

Việc cập nhật có thể được biểu diễn bằng công thức dưới đây:
$$
(\boldsymbol{\omega}, b) \leftarrow (\boldsymbol{\omega}, b) - \frac{\eta}{|\boldsymbol{N}|} \sum_{i \in \boldsymbol{N}} \partial_{(\boldsymbol{\omega}, b)}L_i(\boldsymbol{\omega}, b)
$$
hay,
$$
\boldsymbol{\omega} \leftarrow \boldsymbol{\omega} - \frac{\eta}{|\boldsymbol{N}|} \sum_{i \in \boldsymbol{N}} \partial_(\boldsymbol{\omega}) L_i(\boldsymbol{\omega}, b)
$$
$$
b \leftarrow b - \frac{\eta}{|\boldsymbol{N}|} \sum_{i \in \boldsymbol{N}} \partial_{b} L_i(\boldsymbol{\omega}, b)
$$

Việc cập nhật này cho từng mẫu dữ liệu có thể cực kì chậm. Nếu chúng ta chỉ muốn lấy một minibatch ngẫu nhiên các mẫu mỗi khi ta cần tính bước cập nhật, phương pháp biến thể này là Stochastic Gradient Descent (SGD). Trong mỗi vòng lặp, đầu tiên chúng ta lấy ngẫu nhiên một minibatch $\boldsymbol{B}$ dữ liệu huấn luyện với kích thước cố định. Sau đó, chúng ta tính đạo hàm của hàm mất mát trên minibatch đó theo các tham số của mô hình. Cuối cùng, gradient này được nhân với tốc độ học  $\eta > 0$ và kết quả này được trừ đi từ các giá trị tham số hiện tại.

Việc cập nhật có thể được biểu diễn bằng công thức dưới đây:
$$
(\boldsymbol{\omega}, b) \leftarrow (\boldsymbol{\omega}, b) - \frac{\eta}{|\boldsymbol{B}|} \sum_{i \in \boldsymbol{B}} \partial_{(\boldsymbol{\omega}, b)}L_i(\boldsymbol{\omega}, b)
$$
hay,
$$
\boldsymbol{\omega} \leftarrow \boldsymbol{\omega} - \frac{\eta}{|\boldsymbol{B}|} \sum_{i \in \boldsymbol{B}} \partial_(\boldsymbol{\omega}) L_i(\boldsymbol{\omega}, b)
$$
$$
b \leftarrow b - \frac{\eta}{|\boldsymbol{B}N|} \sum_{i \in \boldsymbol{B}} \partial_{b} L_i(\boldsymbol{\omega}, b)
$$

### Ứng dụng các Thuật toán tối ưu cho Hồi quy Tuyến tính

Để thuận tiện, hàm lập trình GD, SGD và SGD theo minibatch sẽ có danh sách tham số giống nhau. Cụ thể, chúng ta thêm trạng thái đầu vào biến `states` và đặt siêu tham số trong biến `hyperparams`. Bên cạnh đó, chúng ta sẽ tính giá trị mất mát trung bình của từng minibatch trong hàm huấn luyện, từ đó không cần phải chia gradient cho kích thước batch trong thuật toán tối ưu nữa. 

In [ ]:
def sgd(params, states, hyperparams):
    for p in params:
        p.data.sub_(hyperparams['lr'] * p.grad)
        p.grad.data.zero_()

**Giải thích**

Hàm `sgd` sẽ duyệt qua các tham số `p` trong `params` của mô hình, trong bài toán Hồi quy Tuyến tính sẽ là vector trọng số $\boldsymbol{\omega}$ và $b$ và cập nhật theo quy tắc:
$$
p := p - \eta*\Delta_pL
$$
Trong đó:
- `p`: tham số
- `p.grad`: đạo hàm của hàm mất mát theo p
- `hyperparams['lr']` hay $\eta$: tốc độ học (learning rate)
Sau đó, thực hiện reset gradient ở bước cuối trong mỗi vòng lặp để tránh tích lũy gradient từ nhiều batch

Tiếp theo, chúng ta hiện thực một hàm huấn luyện tổng quát, sử dụng được cho tất cả các thuật toán tối ưu. Hàm sẽ khởi tạo một mô hình Hồi quy Tuyến tính và có thể được sử dụng để huấn luyện mô hình với GD, SGD và SGD theo minibatch.

In [ ]:
#@save
def train_ch11(trainer_fn, states, hyperparams, data_iter,
               feature_dim, num_epochs=2):
    # Initialization
    w = torch.normal(mean=0.0, std=0.01, size=(feature_dim, 1),
                     requires_grad=True)
    b = torch.zeros((1), requires_grad=True)
    net, loss = lambda X: d2l.linreg(X, w, b), d2l.squared_loss
    # Train
    animator = d2l.Animator(xlabel='epoch', ylabel='loss',
                            xlim=[0, num_epochs], ylim=[0.22, 0.35])
    n, timer = 0, d2l.Timer()
    for _ in range(num_epochs):
        for X, y in data_iter:
            l = loss(net(X), y).mean()
            l.backward()
            trainer_fn([w, b], states, hyperparams)
            n += X.shape[0]
            if n % 200 == 0:
                timer.stop()
                animator.add(n/X.shape[0]/len(data_iter),
                             (d2l.evaluate_loss(net, data_iter, loss),))
                timer.start()
    print(f'loss: {animator.Y[0][-1]:.3f}, {timer.sum()/num_epochs:.3f} sec/epoch')
    return timer.cumsum(), animator.Y[0]

**Giải thích**

Hàm `train_ch11` sẽ khởi tạo các giá trị cần cho mô hình Hồi quy Tuyến tính và thực hiện huấn luyện.

Tham số đầu vào:
- `trainer_fn`: hàm cập nhật tham số mô hình (GD, SGD, ...)
- `states`: các trạng thái cần thiết cho `trainer_fn`
- `hyperparams`: các siêu tham số như `lr`, `beta`
- `feature_dim`: số lượng đặc trưng (hoặc số chiều biến đầu vào)
- `num_epochs`: số vòng lặp huấn luyện

**Bước 1: Khởi tạo mô hình**
Đầu tiên, khởi tạo 2 vector trọng số $\boldsymbol{\omega}$ và bias $b$ đều yêu cầu gradient, trong đó:
- $\boldsymbol{\omega}$ tuân theo phân phối chuẩn với $\mu$ = 0.0 và $\sigma$ = 0.01, có kích thước `feature_dim` x 1 
- $b$ = 0

Tiếp theo, khởi tạo `net` với `loss`, lần lượt là mô hình Hồi quy tuyến tính có dạng $y = \boldsymbol{X}*\boldsymbol{\omega} + b$ và hàm mất mát theo MSE

**Bước 2: Khởi tạo tiến trình vẽ hàm mất mát và bộ đếm thời gian**
- `animator`: thực thể để biểu diễn hàm mất mát dưới dạng đồ thị theo thời gian
- `n`: tổng số mẫu đã xử lý
- `timer`: để đo thời gian chạy từng epoch

**Bước 3: Huấn luyện mô hình**
Lần lượt duyệt qua từng batch:
1. Tính giá trị hàm mất mát: `l = loss(net(X), y).mean()` tương ứng việc lấy trung bình giá trị MSE cho toàn bộ các điểm dữ liệu trong batch
2. Thực hiện lan truyền ngược để tính GD: `l.backward()`
3. Cập nhật tham số: `trainer_fn` được gọi với `[w, b]`, `states` và `hyperparams`
4. Reset gradient descent nằm trong `trainer_fn`.

Vẽ hàm mất mát với mỗi 200 mẫu

**Bước 4: In kết quả**
In giá trị hàm mất mát cuối cùng và thời gian trung bình mỗi epoch

Tiếp theo, ta tạo một hàm đầu vào để thực hiện toàn quá trình

In [ ]:
def train_sgd(lr, batch_size, num_epochs=2):
    data_iter, feature_dim = get_data_ch11(batch_size)
    return train_ch11(sgd, None, {'lr': lr}, data_iter, feature_dim, num_epochs)

**Giải thích**
Hàm `train_sgd` thực hiện việc đọc dữ liệu, khởi tạo các tham số và huấn luyện mô hình.

Tham số đầu vào:
- `lr`: siêu tham số tốc độ học (learning rate)
- `batch_size`: kích thước của batch
- `num_epochs`: số vòng lặp huấn luyện

Dữ liệu được trả về theo từng batch từ hàm `get_data_ch11` sẽ được dùng để huấn luyện trong hàm `train_ch11`

#### Thực nghiệm với GD
Hãy cùng quan sát quá trình tối ưu của thuật toán Gradient Descent (GD) theo toàn bộ batch. Ta có thể sử dụng toàn bộ batch bằng cách thiết lập kích thước minibatch bằng tổng số mẫu (trong trường hợp này là 1500). Kết quả là các tham số mô hình chỉ được cập nhật một lần duy nhất trong mỗi epoch. Có thể thấy không có tiến triển nào đáng kể. Trong ví dụ, việc tối ưu bị ngừng trệ sau 6 epoch.

In [ ]:
gd_res = train_sgd(1, 1500, 10)

**Giải thích**

Ở đây ta thực nghiệm huấn luyện mô hình với GD, sử dụng $\eta = 1$, số lượng epoch = 10 và thiết lập tham số `batch_size` = 1500 (= kích thước của tập dữ liệu), tức là với mỗi epoch hàm mất mát sẽ được tính và thực hiện cập nhật $(\boldsymbol{\omega}, b)$ chỉ 1 lần duy nhất

#### Thực nghiệm với SGD

Khi kích thước của batch bằng 1, chúng ta sử dụng thuật toán SGD để tối ưu. Để đơn giản hóa việc hiện thực, chúng ta cố định tốc độ học (learning rate) bằng một hằng số (có giá trị nhỏ). Trong SGD, các tham số mô hình được cập nhật bất cứ khi nào có một mẫu huấn luyện được xử lý. Trong trường hợp này, sẽ có 1500 lần cập nhật trong mỗi epoch. Có thể thấy, sự suy giảm giá trị của hàm mục tiêu chậm lại sau một epoch. Mặc dù cả hai thuật toán cùng xử lý 1500 mẫu trong một epoch, SGD tốn thời gian hơn GD trong thí nghiệm trên. Điều này là do SGD cập nhật các tham số thường xuyên hơn và kém hiệu quả khi xử lý đơn lẻ từng mẫu.

In [ ]:
sgd_res = train_sgd(0.005, 1)

**Giải thích**

Ở đây ta thực nghiệm huấn luyện mô hình với SGD, sử dụng $\eta = 0.005$, số lượng epoch = 2 (theo mặc định) và thiết lập tham số `batch_size` = 1, tức là với mỗi epoch hàm mất mát sẽ được tính và thực hiện cập nhật $(\boldsymbol{\omega}, b)$ với mỗi điểm dữ liệu trong tập.

#### Thực nghiệm với SGD theo minibatch

Cuối cùng, khi kích thước của batch bằng 100, chúng ta sử dụng thuật toán SGD theo minibatch để tối ưu. Thời gian cần thiết cho mỗi epoch ngắn hơn thời gian tương ứng của SGD và GD theo toàn bộ batch.

In [ ]:
mini1_res = train_sgd(.4, 100)

**Giải thích**

Ở đây ta thực nghiệm huấn luyện mô hình với SGD theo minibatch có kích thước = 100, sử dụng $\eta = 0.4$, số lượng epoch = 2 (theo mặc định), tức là với mỗi epoch hàm mất mát sẽ được tính và thực hiện cập nhật $(\boldsymbol{\omega}, b)$ tương ứng là $\frac{1500}{100} = 15$ lần

Giảm kích thước của batch bằng 10, thời gian cho mỗi epoch tăng vì thực thi tính toán trên mỗi batch kém hiệu quả hơn.

In [ ]:
mini2_res = train_sgd(.05, 10)

**Giải thích**

Ở đây ta thực nghiệm huấn luyện mô hình với SGD theo minibatch có kích thước = 10, sử dụng $\eta = 0.05$, số lượng epoch = 2 (theo mặc định), tức là với mỗi epoch hàm mất mát sẽ được tính và thực hiện cập nhật $(\boldsymbol{\omega}, b)$ tương ứng là $\frac{1500}{10} = 150$ lần

Cuối cùng, chúng ta so sánh tương quan thời gian và giá trị hàm mấy mát trong bốn thí nghiệm trên. Có thể thấy, dù hội tụ nhanh hơn GD về số mẫu được xử lý, SGD tốn nhiều thời gian hơn để đạt được cùng giá trị mất mát như GD vì thuật toán này tính gradient descent trên từng mẫu một. Thuật toán SGD theo minibatch có thể cân bằng giữa tốc độ hội tụ và hiệu quả tính toán. Với kích thước minibatch bằng 10, thuật toán này hiệu quả hơn SGD; và với kích thước minibatch bằng 100, thời gian chạy của thuật toán này thậm chí nhanh hơn cả GD.

## Hiện thực chính xác

Trong Gluon, chúng ta có thể sử dụng lớp `Trainer` để gọi các thuật toán tối ưu. Cách này được sử dụng để có thể hiện thực một hàm huấn luyện tổng quát. Chúng ta sẽ sử dụng hàm này xuyên suốt các phần tiếp theo của chương.

In [ ]:
#@save
def train_concise_ch11(trainer_fn, hyperparams, data_iter, num_epochs=4):
    # Initialization
    net = nn.Sequential(nn.Linear(5, 1))
    def init_weights(module):
        if type(module) == nn.Linear:
            torch.nn.init.normal_(module.weight, std=0.01)
    net.apply(init_weights)

    optimizer = trainer_fn(net.parameters(), **hyperparams)
    loss = nn.MSELoss(reduction='none')
    animator = d2l.Animator(xlabel='epoch', ylabel='loss',
                            xlim=[0, num_epochs], ylim=[0.22, 0.35])
    n, timer = 0, d2l.Timer()
    for _ in range(num_epochs):
        for X, y in data_iter:
            optimizer.zero_grad()
            out = net(X)
            y = y.reshape(out.shape)
            l = loss(out, y)
            l.mean().backward()
            optimizer.step()
            n += X.shape[0]
            if n % 200 == 0:
                timer.stop()
                # `MSELoss` computes squared error without the 1/2 factor
                animator.add(n/X.shape[0]/len(data_iter),
                             (d2l.evaluate_loss(net, data_iter, loss) / 2,))
                timer.start()
    print(f'loss: {animator.Y[0][-1]:.3f}, {timer.sum()/num_epochs:.3f} sec/epoch')

In [ ]:
data_iter, _ = get_data_ch11(10)
trainer = torch.optim.SGD
train_concise_ch11(trainer, {'lr': 0.01}, data_iter)

## Excercises

### Exercise 1.

In [ ]:
# Thử nghiệm với các giá trị khác nhau của tốc độ học và kích thước batch
# Tốc độ học lớn hơn và kích thước batch nhỏ hơn
experiment_1 = train_sgd(lr=0.1, batch_size=5, num_epochs=5)


In [ ]:
# Tốc độ học nhỏ hơn và kích thước batch lớn hơn
experiment_2 = train_sgd(lr=0.01, batch_size=50, num_epochs=5)


In [ ]:
# Tốc độ học trung bình và kích thước batch trung bình
experiment_3 = train_sgd(lr=0.05, batch_size=20, num_epochs=5)

1. **Tốc độ học lớn hơn và kích thước batch nhỏ hơn**:
    - Với tốc độ học lớn và kích thước batch nhỏ, mô hình có thể hội tụ nhanh hơn nhưng dễ gặp phải dao động lớn trong quá trình tối ưu hóa do phương sai cao của gradient.

2. **Tốc độ học nhỏ hơn và kích thước batch lớn hơn**:
    - Với tốc độ học nhỏ và kích thước batch lớn, mô hình hội tụ ổn định hơn nhưng tốc độ hội tụ có thể chậm hơn do các bước cập nhật nhỏ.

3. **Tốc độ học trung bình và kích thước batch trung bình**:
    - Với tốc độ học và kích thước batch trung bình, mô hình đạt được sự cân bằng giữa tốc độ hội tụ và độ ổn định, thường mang lại kết quả tốt nhất.

Kích thước batch và tốc độ học là các siêu tham số quan trọng, cần được điều chỉnh phù hợp với bài toán cụ thể để đạt hiệu quả tối ưu.

### Exercise 2.

In [ ]:
import mxnet as mx
from mxnet import gluon, nd

# Define a simple model
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(10))
net.initialize(mx.init.Xavier())

# Initialize Trainer with SGD optimizer
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

# Simulate training loop
num_epochs = 5
for epoch in range(num_epochs):
    # Reduce learning rate by a factor of 10 after each epoch
    new_lr = trainer.learning_rate * 0.1
    trainer.set_learning_rate(new_lr)
    print(f'Epoch {epoch+1}: Learning rate = {new_lr}')


### Exercise 3.

Khi huấn luyện mô hình sử dụng phương pháp SGD theo minibatch thông thường, ta có:
- Cách hoạt động:
    + Với mỗi vòng lặp huấn luyện, tập dữ liệu sẽ được chia nhỏ thành các minibatch.
    + Mỗi minibatch chứa các mẫu dữ liệu độc nhất cho đến khi vòng lặp được hoàn thành.
    + Mỗi điểm dữ liệu chỉ được dùng 1 lần mỗi vòng lặp.
- Đặc điểm:
    + Ước lượng gradient hiệu quả: phương sai cho mỗi minibatch tương đối nhỏ.
    + Hội tụ nhanh và mượt mà hơn.
    + Đảm bảo các mẫu đều được xem xét mỗi vòng lặp huấn luyện.

So sánh với biến thể minibatch cho phép lấy mẫu hoàn lại từ tập dữ liệu, ta có:
- Cách hoạt động:
    + Với mỗi vòng lặp huấn luyện, các minibatch sẽ được tạo ra bằng cách lấy ngẫu nhiên từ tập dữ liệu.
    + Một vài mẫu sẽ được chọn nhiều lần, một vài mẫu thì không bao giờ được chọn.
    + Không đảm bảo rằng tất cả các điểm dữ liệu được dùng 1 lần mỗi vòng lặp.
- Đặc điểm:
    + Ước lượng gradient kém hiệu quả: có nhiều nhiễu do bị trùng hoặc thiếu một phần mẫu.
    + Hội tụ chậm hoặc kém ổn định
    + Thiếu tính khái quát hóa do bị overfit với các mẫu dữ liệu được lựa chọn nhiều và underfit với các mẫu dữ liệu không được lựa chọn. 



### Exercise 4.

Nếu tập dữ liệu bị lặp lại, ta đang có kích thước của mẫu quan sát tăng lên nhưng không có thêm thông tin mới. Điều này ảnh hưởng khác nhau đến các phương pháp tối ưu theo các cách khác nhau:
1. Gradient Descent (GD) - Sử dụng toàn bộ mẫu dữ liệu
- Gradient Descent sử dụng toàn bộ mẫu dữ liệu, nên sẽ làm tăng giá trị hàm mất mát và độ dốc theo tỉ lệ thuận. Tuy nhiên, hướng của gradient sẽ không đổi, do ta chỉ cộng các thành phần lặp lại.
- Ảnh hưởng:
    + Hướng của gradient được giữ nguyên, nhưng được tăng lên.
    + Tốc độ học cần phải được điều chỉnh để duy trì được độ ổn định.
    + Chi phí tính toán tăng lên (lâu hơn mỗi vòng lặp nhưng không có thêm thông tin gì)
2. Stochastic Gradient Descent (SGD) – Sử dụng 1 mẫu
- Mỗi mẫu sẽ có khả năng cao hơn được chọn nhiều lần, nhưng không có sự thay đổi về chất lượng gradient và tính đa dạng của tập dữ liệu
- Ảnh hưởng:
    + Không có sự khác biệt cơ bản về quá trình huấn luyện.
    + Phương sai không đổi.
    + Hội tụ chậm hơn do trùng lặp dữ liệu.
3. Minibatch Stochastic Gradient Descent (Minibatch SGD) - Sử dụng tập con
- Các minibatch sẽ chứa nhiều mẫu trùng lặp hơn giữa các vòng lặp huấn luyện.
- Tốc độ học có thể chậm lại do liên tục thấy các mẫu trùng lặp
- Ảnh hưởng:
    + Hướng của gradient được giữ nguyên, nhưng tính khái quát lâu được cải thiện.
    + Có thể tốn các vòng lặp để huấn luyện các mẫu trùng lặp.